In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import torch

from utils import matrl2_error, rl2_error
from ops import injection2d, injection4d 
from ops import interp2d, interp1d_cols, interp1d_rows
from ops import restrict2d
from ops import fetch_nbrs2d #, fetch_nbrs4d
from ops import coord2idx2d, coord2idx4d
from ops import cat2d_nbr_coords
from ops import grid2d_coords, grid4d_coords
from einops import rearrange

from mlmm import Grid2D
from mlmm import K_local_interp_4D, K_local_eval_4D

In [2]:
l = 65
poisson_data = scipy.io.loadmat(f'../pde_data/green_learning/data2d_{l}/poisson.mat')
# A = poisson_data['A']
# U = poisson_data['U']
F = poisson_data['F']
# G = np.linalg.inv(A.todense())
# U_ = torch.tensor(G @ F).float()
# G = torch.tensor(np.array(G).reshape(l,l,l,l)).float()
# GHH = torch.tensor(np.array(G).reshape(l,l,l,l))

# plt.imshow(U[:,40].reshape(l,l)) #, vmin=-0.005, vmax=0.005)
# plt.colorbar()

In [3]:
def kernel_func(pts_pairs):
    x1 = pts_pairs[:,0]
    y1 = pts_pairs[:,1]

    x2 = pts_pairs[:,2]
    y2 = pts_pairs[:,3]

    mask = ((x1**2+y1**2) < 1) & ((x2**2+y2**2) < 1)

    k = 1/(4*torch.pi) * torch.log(((x1 - x2)**2 + (y1-y2)**2) / ((x1*y2-x2*y1)**2 + (x1*x2+y1*y2-1)**2))
    k = torch.nan_to_num(k, neginf=-2) * mask

    return k

In [4]:
def ffunc(pts):
    x = pts[...,0]
    y = pts[...,1]
    u = (1 - (x**2+y**2))**-0.5
    u = torch.nan_to_num(u, posinf=0)
    return u

In [9]:
# two level mm

l= 6
nh = 2**l+1 # total level
nH = 2**(l-1) + 1
k = 1 # coarse level
d = 1 # problem dimension
m = 3 # local range

fine_grid = Grid2D(nh, m)
fine_grid.init_grid_hh()
coarse_grid = Grid2D(nH, m)
coarse_grid.init_grid_hh()

# full eval
f_h = ffunc(fine_grid.x_h).reshape(nh,nh)#torch.tensor(F[:,10]).float().reshape(nh,nh)
K_hh = kernel_func(fine_grid.x_hh.reshape(-1,4))
h = fine_grid.h 
nh = fine_grid.nh
hh = h**2
u_h = hh * (K_hh.reshape(nh*nh, nh*nh) @ f_h.reshape(-1)).reshape(nh,nh)

# coarse eval
H = coarse_grid.h 
nH = coarse_grid.nh
HH = H**2
f_H = restrict2d(f_h[None,None])[0,0]
K_HH = kernel_func(coarse_grid.x_hh.reshape(-1,4))
u_H = HH * (K_HH.reshape(nH*nH, nH*nH) @ f_H.reshape(-1)).reshape(nH,nH)
u_h_ = interp2d(u_H[None,None])[0,0].reshape(-1) # without correction
print("m {:} : {:.4e} ".format(
    2*m, matrl2_error(u_h_, u_h).numpy()))

# local eval
idx_corr = coarse_grid.fetch_local_idx()
x_locals, x_2Ij = coarse_grid.fetch_K_local_x()
K_locals, K_2Ij = K_local_eval_4D(x_2Ij, kernel_func)
K_IJ = K_HH[coarse_grid.ij_idx]
K_IJ = K_IJ.reshape(nH,nH,2*m+1,2*m+1)
K_locals_ = K_local_interp_4D(K_IJ, K_2Ij)

K_corr_even = [(K-K_).reshape(-1) for K, K_ in zip(K_locals[:2], K_locals_[:2])]
K_corr_odd = [(K-K_).reshape(-1) for K, K_ in zip(K_locals[2:], K_locals_[2:])]

idx_corr_even = torch.concat(idx_corr[:2], axis=1)
idx_corr_odd = torch.concat(idx_corr[2:], axis=1)

K_corr_even = torch.concat(K_corr_even, axis=0)
K_corr_odd = torch.concat(K_corr_odd, axis=0)

# local correction
K_corr_even_sparse = torch.sparse_coo_tensor(idx_corr_even, K_corr_even,(nh**2,nh**2))
K_corr_odd_sparse = torch.sparse_coo_tensor(idx_corr_odd, K_corr_odd,(nh**2,nh**2))
u_corr_ = torch.sparse.mm(K_corr_even_sparse, f_h.reshape(-1,1)).reshape(nh,nh)
u_corr_ = injection2d(u_corr_[None,None])[0,0]
u_H_ = u_H + hh * u_corr_
u_h_ = interp2d(u_H_[None,None])[0,0]
print("m {:} : {:.4e} ".format(
    2*m, matrl2_error(u_h_, u_h).numpy()))
u_corr_ = torch.sparse.mm(K_corr_odd_sparse, f_h.reshape(-1,1)).reshape(nh,nh)
u_h_ = u_h_ + hh*u_corr_
print("m {:} : {:.4e} ".format(
    2*m, matrl2_error(u_h_, u_h).numpy()))

m 6 : 5.1386e-02 
m 6 : 1.4147e-02 
m 6 : 8.6846e-03 


In [11]:
%%timeit 

K_hh = kernel_func(fine_grid.x_hh.reshape(-1,4))
u_h = hh * (K_hh.reshape(nh*nh, nh*nh) @ f_h.reshape(-1)).reshape(nh,nh)

227 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit 

K_HH = kernel_func(coarse_grid.x_hh.reshape(-1,4))

x_locals, x_2Ij = coarse_grid.fetch_K_local_x()
K_locals, K_2Ij = K_local_eval_4D(x_2Ij, kernel_func)
K_IJ = K_HH[coarse_grid.ij_idx]
K_IJ = K_IJ.reshape(nH,nH,2*m+1,2*m+1)
K_locals_ = K_local_interp_4D(K_IJ, K_2Ij)
K_corr_even = [(K-K_).reshape(-1) for K, K_ in zip(K_locals[:2], K_locals_[:2])]
K_corr_odd = [(K-K_).reshape(-1) for K, K_ in zip(K_locals[2:], K_locals_[2:])]
idx_corr_even = torch.concat(idx_corr[:2], axis=1)
idx_corr_odd = torch.concat(idx_corr[2:], axis=1)
K_corr_even = torch.concat(K_corr_even, axis=0)
K_corr_odd = torch.concat(K_corr_odd, axis=0)
K_corr_even_sparse = torch.sparse_coo_tensor(idx_corr_even, K_corr_even,(nh**2,nh**2))
K_corr_odd_sparse = torch.sparse_coo_tensor(idx_corr_odd, K_corr_odd,(nh**2,nh**2))

u_H = HH * (K_HH.reshape(nH*nH, nH*nH) @ f_H.reshape(-1)).reshape(nH,nH)
u_corr_ = torch.sparse.mm(K_corr_even_sparse, f_h.reshape(-1,1)).reshape(nh,nh)
u_corr_ = u_corr_[::2,::2]
u_H_ = u_H + hh * u_corr_
u_h_ = interp2d(u_H_[None,None])[0,0]
u_corr_ = torch.sparse.mm(K_corr_odd_sparse, f_h.reshape(-1,1)).reshape(nh,nh)
u_h_ = u_h_ + hh*u_corr_

58.9 ms ± 721 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
